# Build a RAG chain using Zilliz cloud and AWS bedrock
Combine [Zilliz cloud](https://zilliz.com/cloud) and [Aws bedrock](https://aws.amazon.com/bedrock/) to build a RAG(Retrieval-Augmented Generation) chain through [Langchain](https://langchain.com/) framework.

The RAG chain consists of a retriever for retrieving relevant documents from a vector store, a prompt template for generating the input prompt, a language model for generating the AI response, and an output parser for formatting the generated response.

Zilliz cloud is used for vector storage and retrieval, and AWS bedrock is used for supporting language models and embedding models. Langchain is used to connect the components and build the RAG chain.

## Prerequisites
Install the required packages and set the required environment variables.

In [1]:
# ! pip install --upgrade --quiet  langchain langchain-core langchain-text-splitters langchain-community langchain-aws bs4 boto3

In [2]:
import os
import bs4
import boto3
from langchain_aws import ChatBedrock
from langchain_community.vectorstores import Zilliz
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import BedrockEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

Set the required environment variables.

In [3]:
# Set the AWS region and access key environment variables
REGION_NAME = "us-east-1"
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# Set ZILLIZ cloud environment variables
ZILLIZ_CLOUD_URI = os.getenv("ZILLIZ_CLOUD_URI")
ZILLIZ_CLOUD_API_KEY = os.getenv("ZILLIZ_CLOUD_API_KEY")

The zilliz cloud uri and zilliz api key can be obtained from the [Zilliz cloud console guide](https://docs.zilliz.com/docs/on-zilliz-cloud-console).

In simple terms, you can access them on your zilliz cloud cluster page.
 ![](../../images/zilliz_uri_and_key.png)

## Create LLM and Embedding models using aws bedrock
Create an aws bedrock instance and deploy language models and embedding models.

In [4]:
# Create a boto3 client with the specified credentials
client = boto3.client(
    "bedrock-runtime",
    region_name=REGION_NAME,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

# Initialize the ChatBedrock instance for language model operations
llm = ChatBedrock(
    client=client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=REGION_NAME,
    model_kwargs={"temperature": 0.1},
)

# Initialize the BedrockEmbeddings instance for handling text embeddings
embeddings = BedrockEmbeddings(client=client, region_name=REGION_NAME)

## Load documents and split them into chunks
We use the Langchain WebBaseLoader to load documents from web sources and split them into chunks using the RecursiveCharacterTextSplitter.

In [5]:
# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

## Create the RAG chain and invoke it
We use the Langchain framework to create the RAG chain. The RAG chain consists of a retriever for retrieving relevant documents from a Zilliz vector store. When invoke `from_documents` function, the retriever will automatically create a Zilliz vector store from the loaded documents and embeddings.

In [6]:
# Define the prompt template for generating AI responses
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
# Create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

# Initialize Zilliz vector store from the loaded documents and embeddings
vectorstore = Zilliz.from_documents(
    documents=docs,
    embedding=embeddings,
    connection_args={
        "uri": ZILLIZ_CLOUD_URI,
        "token": ZILLIZ_CLOUD_API_KEY,
        "secure": True,
    },
    auto_id=True,
    drop_old=True,
)

# Create a retriever for document retrieval and generation
retriever = vectorstore.as_retriever()


# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
# Define the RAG (Retrieval-Augmented Generation) chain for AI response generation
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.get_graph().print_ascii()

# Invoke the RAG chain with a specific question and retrieve the response
res = rag_chain.invoke("What is self-reflection of an AI Agent?")
print(res)

Self-reflection is a vital capability that allows autonomous AI agents to improve iteratively by analyzing and refining their past actions, decisions, and mistakes. Some key aspects of self-reflection for AI agents include:

1. Evaluating the efficiency and effectiveness of past reasoning trajectories and action sequences to identify potential issues like inefficient planning or hallucinations (generating consecutive identical actions without progress).

2. Synthesizing observations and memories from past experiences into higher-level inferences or summaries to guide future behavior.

3. Generating reflective questions based on recent observations and attempting to answer those questions to gain insights.

4. Incorporating reflections into the agent's working memory to provide additional context for querying the language model and adjusting future plans.

For example, in the Reflexion framework (Shinn & Labash 2023), the agent computes a heuristic after each action to determine if the 

Looks great, the RAG chain answered the question through using relevant knowledge context from the web page and provide concise and correctly answers.